# Double Pendulum Simulation
### by Ganden Schaffner and Ian McDougall

In [1]:
import math
from math import pi
import numpy as np
from numpy import sin, cos, pi
import matplotlib.pyplot as plt
from matplotlib import animation, rc
from IPython.display import HTML
import ipywidgets as widg
%matplotlib inline

# Maths from Matplotlib example bank (https://matplotlib.org/examples/animation/double_pendulum_animated.html)

# Constants
G =  9.81     # acceleration due to gravity (in m/s^2)
L1 = 1.0      # length of pendulum (1 in m)
L2 = 1.0      # length of pendulum 2 (in m)
M1 = 1.0      # mass of pendulum 1 (in kg)
M2 = 1.0      # mass of pendulum 2 (in kg)
rad = pi/180  # radians in 1 deg

# Set initial conditions
th1 = 120.0  # th1 and th2 are the initial angles (degrees)
th2 = -10.0
w1 = 0.0     # w10 and w20 are the initial angular velocities (degrees per second)
w2 = 0.0
state = np.array([th1, w1, th2, w2])*rad  # initial state array

def derivs(state, t):
    dxdt = np.zeros_like(state) # initialize dervative array (values correspond to the time derivatives of initial state array)
    th_diff_init = state[2] - state[0] # difference of angle1 and angle2

    dxdt[0] = state[1] # derivative of angle1

    denom1 = (M1+M2)*L1 - M2*L1*cos(th_diff_init)*cos(th_diff_init) # denominator of dxdt[2]
    dxdt[1] = (M2*L1*state[1]*state[1]*sin(th_diff_init)*cos(th_diff_init)
               + M2*G*sin(state[2])*cos(th_diff_init)
               + M2*L2*state[3]*state[3]*sin(th_diff_init)
               - (M1+M2)*G*sin(state[0]))/denom1 # time derivative of angular velocity 1

    dxdt[2] = state[3] # time derivative of angle2

    denom2 = (L2/L1)*denom1 # denominator of dxdt[3]
    dxdt[3] = (-M2*L2*state[3]*state[3]*sin(th_diff_init)*cos(th_diff_init)
               + (M1+M2)*G*sin(state[0])*cos(th_diff_init)
               - (M1+M2)*L1*state[1]*state[1]*sin(th_diff_init)
               - (M1+M2)*G*sin(state[2]))/denom2   # time derivative of angular velocity 2
    
    return dxdt # return the array of time derivatives
